# CSV creation for Maximum Amplitude Heatplots
This script performs the intermediary step of extracting amplitude and SNR information from the stacked XCORR files. These are saved to CSVs and whch are then read by the plotting script to make heatplots. This uses some of the functions from the module on C4, should be easy to either copy those files and change filepathing or just copy the few needed functions from github into the notebook.

In [ ]:
include("/Users/julianschmitt/Documents/Schoolwork/Seismology/C4-Project.jl/src/SeisCore.jl")
using .SeisCore, SeisIO, SeisNoise, Plots, Dates, CSV, DataFrames, JLD2, Statistics, Glob, ColorSchemes, 
            Plots.PlotUtils, HDF5, Images, Statistics, AbstractFFTs

In [4]:
cd("/")
cd("/Volumes/T7/seis_data/final_stack")
files = glob("*/*")
sources = unique([convert(String, split(split(f,".")[2],"_")[1]) for f in files]);

In [5]:
all_stations = DataFrame(CSV.File("/Users/julianschmitt/Downloads/updated_sources.csv"))
function LLE_geo(station, df)
    """ Find station matching location and return geoloc object"""
    try
        row = df[(findfirst(x -> x==station, df.station)),:]
        lat, lon, el = row.latitude[1], row.longitude[1], row.elevation[1]
        geo = GeoLoc(lat = float(lat), lon = float(lon), el = float(el))
        return geo
    catch 
        return nothing
    end
end
function loc_max_amp2(corr::CorrData, filt::Array{Float64,1}=[0.1,0.2])
    """ Computes essential correlation metrics:
    latitude, longitude, max amplitude, max amplitude for anticausal side,
     max amplitude for causal side, SNR (signal-to-noise ratio), SNR (anticausal side), 
        SNR (causal), correlation name
    """
    lat, lon, dist = corr.loc.lat, corr.loc.lon, corr.dist
    corr = bandpass(fftderivative(fftderivative(corr)),filt[1],filt[2])

    # select windows for waveform and coda - specific to the current processing script
    causal = Int(round(6000+20*dist/5)):Int(round(6000+20*dist/1.5))
    anticausal = Int(round(6000-20*dist/1.5)):Int(round(6000-20*dist/5))
    # select one minute of coda 
    coda_causal = Int(round(6000+20*dist/1.5)):minimum([Int(round(6000+20*dist/1.5)+1200), 12001])
    coda_anti = Int(round(6000-20*dist/1.5)):minimum([Int(round(6000-20*dist/1.5)+1200), 12001])
    
    # select different subsections of waveform for snr and max amplitude calculations
    bpass_corr_both = corr.corr[vcat(causal, anticausal)]
    bpass_noise_anti = corr.corr[anticausal]
    bpass_noise_causal = corr.corr[causal]
    bpass_coda_anti = corr.corr[coda_anti]
    bpass_coda_causal = corr.corr[coda_causal]

    # compute max amplitudes
    #half_len = convert(Int64,round(length(bpass_corr)/2))
    max_ampl = maximum(abs.(bpass_corr_both))
    max_ampl_anti = maximum(abs.(bpass_noise_anti))
    max_ampl_causal = maximum(abs.(bpass_noise_causal))
    
    # compute SNR
    snr = maximum(abs.(bpass_corr_both))/std(vcat(bpass_coda_anti, bpass_coda_causal))
    snr_anti = maximum(abs.(bpass_noise_anti))/std(bpass_coda_anti)
    snr_causal = maximum(abs.(bpass_noise_causal))/std(bpass_coda_causal)
    
    return Dict("lat" => lat, "lon" => lon, "max_amp" => max_ampl, "max_amp_anti" => max_ampl_anti, 
        "max_amp_causal" => max_ampl_causal, "SNR" => snr, "SNR_anticausal" => snr_anti, 
        "SNR_causal" => snr_causal, "station" => corr.name)
end

loc_max_amp2 (generic function with 2 methods)

In [34]:
r = loc_max_amp2(tcorrs[1])
#maximum([Int(round(6000+20*10/1.5)+1200), 12000])

5321:6521


Dict{String,Any} with 9 entries:
  "lat"             => 34.1583
  "max_ampl"        => 1.13297e-11
  "SNR_anticausal"  => 2.56044
  "max_ampl_anti"   => 1.13297e-11
  "SNR_causal"      => 3.99595
  "SNR"             => 3.49418
  "station"         => "NO.B4003"
  "lon"             => -117.647
  "max_ampl_causal" => 4.8184e-12

In [6]:
year1, year2, year3 = 2017, 2018, 2019 # large stacked files have data between 2017 and 2019
filt = "" # return nodal plots
bandpass_filter, stacktype, components = [0.1, 0.35], "linear", ["ZZ","ZN","ZE","NZ","NN","NE","EZ","EN","EE"]
rootdir = "/Volumes/T7/seis_data/"
csv_dir = "/Users/julianschmitt/Downloads/Seismo/amplitude_data"
for source_station in sources#setdiff(Set(sources), Set(["CHN","CJM","DEV","IPT","LPC","SNO","SVD","TA2"]))
    try
        fpath = joinpath(rootdir,"final_stack/2017/CI.$(source_station)_2017.h5")
        f2017 = h5open(fpath,"r")
        f2018 = h5open(joinpath(rootdir,"final_stack/2018/CI.$(source_station)_2018.h5"),"r")
        f2019 = h5open(joinpath(rootdir,"final_stack/2019/CI.$(source_station)_2019.h5"),"r")
        for comp in components
            all_corrs = collect(Iterators.flatten([get_corrs(file, stacktype, comp, filt) for file in [f2017, f2018, f2019]]))
            amplitudes = [loc_max_amp2(corr, bandpass_filter) for corr in all_corrs]
            #pgv_amp = DataFrame(amplitudes, [:lat, :lon, :max_amp, :max_amp_anti, :max_amp_causal, :SNR, :SNR_anticausal, :SNR_causal, :station])#["lat", "lon", "max_amp", "max_amp_anti", "max_amp_causal", "SNR", "SNR_anticausal", "SNR_causal", "station"])#
            # if there's a better way to make this dataframe like ^^ please please please tell me :) 
#             pgd_amp = DataFrame(lat = [x[1] for x in amplitudes], lon = [x[2] for x in amplitudes],
#                 max_amp = [x[3] for x in amplitudes], max_amp_anti = [x[4] for x in amplitudes], 
#                 max_amp_causal = [x[5] for x in amplitudes],SNR = [x[6] for x in amplitudes], 
#                 SNR_anticausal = [x[7] for x in amplitudes], SNR_causal = [x[8] for x in amplitudes], station = [x[end] for x in amplitudes]);
            pgv_amp = DataFrame(lat = Float64[], lon = Float64[], max_amp = Float64[], max_amp_anti = Float64[], 
                        max_amp_causal = Float64[], SNR = Float64[], SNR_anticausal = Float64[], 
                        SNR_causal = Float64[], station = String[])
            map(x -> push!(pgv_amp, x), amplitudes)
            pgv_grouped = aggregate(groupby(pgv_amp, :station), mean)
            #println(pgv_grouped)
            #println(pgd_amp)
            save_path = joinpath(rootdir,"final_maxamp_csvs/PGV_$(bandpass_filter[1])_$(bandpass_filter[2])/PGV_$(source_station)_$comp.csv")
            if !isdir(dirname(save_path))
                mkpath(dirname(save_path))
            end
            CSV.write(save_path ,pgv_grouped)
        end
    catch e
        println(e)
    end
end

HDF5-DIAG: Error detected in HDF5 (1.10.4) thread 0:
  #000: H5O.c line 120 in H5Oopen(): unable to open object
    major: Object header
    minor: Can't open object
  #001: H5Oint.c line 596 in H5O__open_name(): unable to open object
    major: Object header
    minor: Can't open object
  #002: H5Oint.c line 551 in H5O_open_name(): object not found
    major: Object header
    minor: Object not found
  #003: H5Gloc.c line 422 in H5G_loc_find(): can't find object
    major: Symbol table
    minor: Object not found
  #004: H5Gtraverse.c line 851 in H5G_traverse(): internal path traversal failed
    major: Symbol table
    minor: Object not found
  #005: H5Gtraverse.c line 627 in H5G__traverse_real(): traversal operator failed
    major: Symbol table
    minor: Callback failed
  #006: H5Gloc.c line 378 in H5G__loc_find_cb(): object 'ZZ' doesn't exist
    major: Symbol table
    minor: Object not found
HDF5-DIAG: Error detected in HDF5 (1.10.4) thread 0:
  #000: H5O.c line 120 in H5Oopen(

In [15]:
test = h5open(joinpath(rootdir,"nodestack_new/2017/CI.SVD_2017.h5"), "r")
tcorrs = get_corrs(test)

HDF5-DIAG: Error detected in HDF5 (1.10.4) thread 0:
  #000: H5O.c line 120 in H5Oopen(): unable to open object
    major: Object header
    minor: Can't open object
  #001: H5Oint.c line 596 in H5O__open_name(): unable to open object
    major: Object header
    minor: Can't open object
  #002: H5Oint.c line 551 in H5O_open_name(): object not found
    major: Object header
    minor: Object not found
  #003: H5Gloc.c line 422 in H5G_loc_find(): can't find object
    major: Symbol table
    minor: Object not found
  #004: H5Gtraverse.c line 851 in H5G_traverse(): internal path traversal failed
    major: Symbol table
    minor: Object not found
  #005: H5Gtraverse.c line 627 in H5G__traverse_real(): traversal operator failed
    major: Symbol table
    minor: Callback failed
  #006: H5Gloc.c line 378 in H5G__loc_find_cb(): object 'ZZ' doesn't exist
    major: Symbol table
    minor: Object not found
HDF5-DIAG: Error detected in HDF5 (1.10.4) thread 0:
  #000: H5O.c line 120 in H5Oopen(

194-element Array{CorrData,1}:
 CorrData with 0 Corrs
      NAME: "NO.B4003"                         
        ID: ""                                 
       LOC: 34.1583 N, -117.647 E, 649.0 m
      COMP: ""                                 
   ROTATED: false                              
 CORR_TYPE: ""                                 
        FS: 20.0
      GAIN: 1.0
   FREQMIN: 0.0
   FREQMAX: 0.0
    CC_LEN: 0.0
   CC_STEP: 0.0
  WHITENED: false                              
 TIME_NORM: ""                                 
      RESP: a0 1.0, f0 1.0, 0z, 0p
      MISC: 0 entries                          
     NOTES: 0 entries                          
      DIST: 50.9404
       AZI: 276.638
       BAZ: 96.3301
    MAXLAG: 0.0
         T: Float64[]                          
      CORR: 12001×1 Array{Float32,2}           

 CorrData with 0 Corrs
      NAME: "NO.B4004"                         
        ID: ""                                 
       LOC: 34.1567 N, -117.649 E, 628.0 m
    

,lat,lon,max_amp,max_amp_anti,max_amp_causal,SNR,SNR_anticausal
,Float64,Float64,Float64,Float64,Float64,Float64,Float64


In [18]:
typeof(tcorrs[1])

CorrData